<a href="https://colab.research.google.com/github/Koussailakadi/Speech_Input_for_Emotion_Recognition/blob/main/Linguistic_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys

from google.colab import drive 
drive.mount('/content/drive')

import numpy as np 
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def read_text(path):
    
    with open(path) as file:
        text = file.read().splitlines()
        
    return text 

In [5]:
path = '/content/drive/My Drive/Dataset_text'

def read_all_text_file(path):
    os.chdir(path)
    
    text = []
    for file in os.listdir():
        # Check whether file is in text format or not
        if file.endswith(".txt"):
            
            file_path = f"{path}/{file}"
            # call read text file function
            text.append( read_text(file_path) )
            
    return text

In [6]:
datas = read_all_text_file(path)

In [7]:
datas[0]

['7850-73752-0000 FERDINAND MEDITATES OVER HIS GOOD FORTUNE',
 '7850-73752-0001 IN MOMENTS OF DEEP FEELING ALIKE SUDDEN BURSTS OF PROSPERITY AS IN DARKER HOURS MAN MUST BE ALONE',
 '7850-73752-0002 IT REQUIRES SOME SELF COMMUNION TO PREPARE OURSELVES FOR GOOD FORTUNE AS WELL AS TO ENCOUNTER DIFFICULTY AND DANGER AND DISGRACE',
 '7850-73752-0003 THIS VIOLENT AND TRIUMPHANT REVOLUTION IN HIS PROSPECTS AND HIS FORTUNES WAS HARDLY YET COMPLETELY COMPREHENDED BY OUR FRIEND FERDINAND ARMINE AND WHEN HE HAD LEFT A NOTE FOR THE GENEROUS MIRABEL WHOSE SLUMBERS HE WOULD NOT DISTURB AT THIS EARLY HOUR EVEN WITH GOOD NEWS HE STROLLED ALONG UP CHARLES STREET AND TO THE PARK IN ONE OF THOSE WILD AND JOYOUS REVERIES IN WHICH WE BROOD OVER COMING BLISS AND CREATE A THOUSAND GLORIOUS CONSEQUENCES',
 '7850-73752-0004 FERDINAND FELT HIS FREEDOM AS WELL AS HIS HAPPINESS',
 '7850-73752-0005 IT WAS INDEED HER HANDWRITING',
 '7850-73752-0006 RESTLESS WITH IMPENDING JOY HE SAUNTERED TO THE BRIDGE AND LEANT OV

In [8]:
def process_datas(datas) : 
    
    for text in range(len(datas)) :

        for sentence in range(len(datas[text])) :

            datas[text][sentence] = ' '.join(datas[text][sentence].split()[1:])
            
    return datas

In [10]:
datas = process_datas(datas)

In [11]:
classes = ['Negative' , 'Neural' , 'Positive']

In [12]:
!pip install transformers
!pip install --upgrade tensorflow

In [13]:
from transformers import AutoTokenizer, AutoModel, pipeline
tokenizer = AutoTokenizer.from_pretrained("avichr/heBERT_sentiment_analysis") #same as 'avichr/heBERT' tokenizer
model = AutoModel.from_pretrained("avichr/heBERT_sentiment_analysis")


sentiment_analysis = pipeline(
    "sentiment-analysis",
    model="avichr/heBERT_sentiment_analysis",
    tokenizer="avichr/heBERT_sentiment_analysis",
    return_all_scores = True
)


# Exemple 1 : datas[0], first text
predicted_sentence_emotions = sentiment_analysis(datas[0])

Some weights of the model checkpoint at avichr/heBERT_sentiment_analysis were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
predicted_sentence_emotions

[[{'label': 'neutral', 'score': 0.0002908922906499356},
  {'label': 'positive', 'score': 0.0005749870906583965},
  {'label': 'negative', 'score': 0.9991341233253479}],
 [{'label': 'neutral', 'score': 0.00030456596869044006},
  {'label': 'positive', 'score': 0.0002278289175592363},
  {'label': 'negative', 'score': 0.999467670917511}],
 [{'label': 'neutral', 'score': 0.7124680280685425},
  {'label': 'positive', 'score': 0.005660926923155785},
  {'label': 'negative', 'score': 0.2818710505962372}],
 [{'label': 'neutral', 'score': 0.002475605346262455},
  {'label': 'positive', 'score': 0.0007221829146146774},
  {'label': 'negative', 'score': 0.9968022108078003}],
 [{'label': 'neutral', 'score': 0.00023707616492174566},
  {'label': 'positive', 'score': 0.00034564550151117146},
  {'label': 'negative', 'score': 0.9994173049926758}],
 [{'label': 'neutral', 'score': 0.0007437324384227395},
  {'label': 'positive', 'score': 0.9990192651748657},
  {'label': 'negative', 'score': 0.000237022220971994

## In the variable predicted_sentence_emotions, we can find the sentiment prediction for each sentence (3 classes : positive, negative and neural)

In [15]:
def prediction_class(predicted_sentence_emotions) :
  prediction = []
  for predict in range(len(predicted_sentence_emotions)) :
    score = []
    for i in range(len(predicted_sentence_emotions[predict])) : 
      score.append( predicted_sentence_emotions[predict][i]['score'] )
    indice = np.where(np.array(score) == np.amax(np.array(score)))
    prediction.append( predicted_sentence_emotions[predict][indice[0][0]]['label'] )

  return prediction

In [16]:
prediction = prediction_class(predicted_sentence_emotions) # contains the prediction for each sentence in the text
prediction

['negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral']

In [17]:
def global_label(prediction):
  positive = prediction.count('positive')
  negative = prediction.count('negative')
  neural = prediction.count('neural')

  if positive > negative > neural :
    return 'positive'
  elif negative > positive > neural :
    return 'negative'
  else :
    return 'neural'

In [18]:
global_label(prediction)

'negative'

In [19]:
class_global = []
for i in range(10) :
  predicted_sentence_emotions = sentiment_analysis(datas[i])
  prediction = prediction_class(predicted_sentence_emotions)
  class_global.append( global_label(prediction) )


In [20]:
class_global # model prediction

['negative',
 'negative',
 'negative',
 'neural',
 'negative',
 'neural',
 'negative',
 'negative',
 'negative',
 'neural']

In [22]:
os.chdir('/')
df = pd.read_csv("text.csv") # We add real target for 10 exemples for evaluate the model prediction

In [23]:
df[:11]

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,text
NaN,Target,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89
0.0,positive,FERDINAND MEDITATES OVER HIS GOOD FORTUNE,IN MOMENTS OF DEEP FEELING ALIKE SUDDEN BURSTS OF PROSPERITY AS IN DARKER HOURS MAN MUST BE ALONE,IT REQUIRES SOME SELF COMMUNION TO PREPARE OURSELVES FOR GOOD FORTUNE AS WELL AS TO ENCOUNTER DIFFICULTY AND DANGER AND DISGRACE,THIS VIOLENT AND TRIUMPHANT REVOLUTION IN HIS PROSPECTS AND HIS FORTUNES WAS HARDLY YET COMPLETELY COMPREHENDED BY OUR FRIEND FERDINAND ARMINE AND WHEN HE HAD LEFT A NOTE FOR THE GENEROUS MIRABEL WHOSE SLUMBERS HE WOULD NOT DISTURB AT THIS EARLY HOUR EVEN WITH GOOD NEWS HE STROLLED ALONG UP CHARLES STREET AND TO THE PARK IN ONE OF THOSE WILD AND JOYOUS REVERIES IN WHICH WE BROOD OVER COMING BLISS AND CREATE A THOUSAND GLORIOUS CONSEQUENCES,FERDINAND FELT HIS FREEDOM AS WELL AS HIS HAPPINESS,IT WAS INDEED HER HANDWRITING,RESTLESS WITH IMPENDING JOY HE SAUNTERED TO THE BRIDGE AND LEANT OVER THE BALUSTRADE GAZING ON THE WATERS IN CHARMED AND CHARMING VACANCY,HOW MANY INCIDENTS HOW MANY CHARACTERS HOW MANY FEELINGS FLITTED OVER HIS MEMORY OF WHAT SWEET AND BITTER EXPERIENCE DID HE NOT CHEW THE CUD,FOUR AND TWENTY HOURS AGO AND HE DEEMED HIMSELF THE MOST MISERABLE AND FORLORN OF HUMAN BEINGS AND NOW ALL THE BLESSINGS OF THE WORLD SEEMED SHOWERED AT HIS FEET,THE MOST GIFTED INDIVIDUALS IN THE LAND EMULATED EACH OTHER IN PROVING WHICH ENTERTAINED FOR HIM THE MOST SINCERE AFFECTION,HE COULD NOT FLATTER HIMSELF THAT HE INDEED MERITED SUCH SINGULAR BLESSINGS AND YET WITH ALL HIS FAULTS WHICH WITH HIM WERE BUT THE CONSEQUENCES OF HIS FIERY YOUTH FERDINAND HAD BEEN FAITHFUL TO HENRIETTA,HIS CONSTANCY TO HER WAS NOW REWARDED,AS FOR HIS FRIENDS THE FUTURE MUST PROVE HIS GRATITUDE TO THEM,IT WAS INDEED DANCING ON A VOLCANO,AND NOW ALL HAD ENDED SO HAPPILY,WAS IT NOT ALL A DREAM OF HIS OWN CREATION WHILE HIS EYE HAD BEEN FIXED IN ABSTRACTION ON THAT BRIGHT AND FLOWING RIVER,HE MIGHT BE ENCHANTED BUT THAT WAS THE TALISMAN,IN THE PRESENT UNSETTLED THOUGH HOPEFUL STATE OF AFFAIRS FERDINAND WOULD NOT GO HOME,IN EXACTLY TEN MINUTES IT IS IN THE POWER OF EVERY MAN TO FREE HIMSELF FROM ALL THE TUMULT OF THE WORLD THE PANGS OF LOVE THE THROBS OF AMBITION THE WEAR AND TEAR OF PLAY THE RECRIMINATING BOUDOIR THE CONSPIRING CLUB THE RATTLING HELL AND FIND HIMSELF IN A SUBLIME SYLVAN SOLITUDE SUPERIOR TO THE CEDARS OF LEBANON AND INFERIOR ONLY IN EXTENT TO THE CHESTNUT FORESTS OF ANATOLIA,IS PAPA ALONE ENQUIRED MISS TEMPLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,positive,THROUGH THE INFLUENCE OF HON THOMAS L HAMER HE WAS ADMITTED AT WEST POINT IN EIGHTEEN THIRTY NINE,AT THIS TIME GRANT WAS NOT TAKEN WITH WAR AND PROBABLY EVINCED LITTLE INTEREST IN ARMY TACTICS,GRANT ACTED AS MUSTERING OFFICER UNTIL BEING COMMISSIONED COLONEL OF THE TWENTY FIRST ILLINOIS VOLUNTEERS HE TOOK THE FIELD,GENERAL HALLECK IN SPEAKING OF THIS BATTLE SAID,INDEED IF EVER A GENERAL DESERVED HONOR GRANT HAD WON IT HE HAD OPENED THE MISSISSIPPI TO NAVIGATION AND HAD CAPTURED NEARLY ONE HUNDRED THOUSAND PRISONERS AND ARMS,HE WAS NOW COMMANDER OF ALL THE FEDERAL FORCES,THE CAPTURE OF LEE WAS A FAR MORE DIFFICULT UNDERTAKING,TIME WORE AWAY AND ON THE NINTH OF APRIL EIGHTEEN SIXTY FIVE GRANT CAPTURED THE CONFEDERATE ARMY UNDER LEE THUS VIRTUALLY ENDING THE WAR,WHEN HIS PUBLIC SERVICES WERE FINISHED HE STARTED IN COMPANY WITH HIS WIFE SON JESSE AND A FEW FRIENDS,HIS SUCCESS SEEMS TO HAVE BEEN THE OUTGROWTH OF HARD 